In [ ]:
import gurobipy as gp
from gurobipy import *

days=[0,1,2,3,4,5,6,7,8,9,10,11,12,13]

schedule = Model ()


#All possible combinations of games nxn (first digit == Home game || second digit = Away game)
#Combinations X(i,j) where i=j (e.g. X11) are always equal to zero 0 (a team cannot play against itself)

X12=schedule.addVars( days , vtype=GRB.BINARY, name='T1vsT2')
X21=schedule.addVars( days , vtype=GRB.BINARY, name='T2vsT1')

X13=schedule.addVars( days , vtype=GRB.BINARY, name='T1vsT3')
X31=schedule.addVars( days , vtype=GRB.BINARY, name='T3vsT1')

X14=schedule.addVars( days , vtype=GRB.BINARY, name='T1vsT4')
X41=schedule.addVars( days , vtype=GRB.BINARY, name='T4vsT1')

X15=schedule.addVars( days , vtype=GRB.BINARY, name='T1vsT5')
X51=schedule.addVars( days , vtype=GRB.BINARY, name='T5vsT1')

X16=schedule.addVars( days , vtype=GRB.BINARY, name='T1vsT6')
X61=schedule.addVars( days , vtype=GRB.BINARY, name='T6vsT1')

X17=schedule.addVars( days , vtype=GRB.BINARY, name='T1vsT7')
X71=schedule.addVars( days , vtype=GRB.BINARY, name='T7vsT1')

X18=schedule.addVars( days , vtype=GRB.BINARY, name='T1vsT8')
X81=schedule.addVars( days , vtype=GRB.BINARY, name='T8vsT1')

X23=schedule.addVars( days , vtype=GRB.BINARY, name='T2vsT3')
X32=schedule.addVars( days , vtype=GRB.BINARY, name='T3vsT2')

X24=schedule.addVars( days , vtype=GRB.BINARY, name='T2vsT4')
X42=schedule.addVars( days , vtype=GRB.BINARY, name='T4vsT2')

X25=schedule.addVars( days , vtype=GRB.BINARY, name='T2vsT5')
X52=schedule.addVars( days , vtype=GRB.BINARY, name='T5vsT2')

X26=schedule.addVars( days , vtype=GRB.BINARY, name='T2vsT6')
X62=schedule.addVars( days , vtype=GRB.BINARY, name='T6vsT2')

X27=schedule.addVars( days , vtype=GRB.BINARY, name='T2vsT7')
X72=schedule.addVars( days , vtype=GRB.BINARY, name='T7vsT2')

X28=schedule.addVars( days , vtype=GRB.BINARY, name='T2vsT8')
X82=schedule.addVars( days , vtype=GRB.BINARY, name='T8vsT2')

X34=schedule.addVars( days , vtype=GRB.BINARY, name='T3vsT4')
X43=schedule.addVars( days , vtype=GRB.BINARY, name='T4vsT3')

X35=schedule.addVars( days , vtype=GRB.BINARY, name='T3vsT5')
X53=schedule.addVars( days , vtype=GRB.BINARY, name='T5vsT3')

X36=schedule.addVars( days , vtype=GRB.BINARY, name='T3vsT6')
X63=schedule.addVars( days , vtype=GRB.BINARY, name='T6vsT3')

X37=schedule.addVars( days , vtype=GRB.BINARY, name='T3vsT7')
X73=schedule.addVars( days , vtype=GRB.BINARY, name='T7vsT3')

X38=schedule.addVars( days , vtype=GRB.BINARY, name='T3vsT8')
X83=schedule.addVars( days , vtype=GRB.BINARY, name='T8vsT3')

X45=schedule.addVars( days , vtype=GRB.BINARY, name='T4vsT5')
X54=schedule.addVars( days , vtype=GRB.BINARY, name='T5vsT4')

X46=schedule.addVars( days , vtype=GRB.BINARY, name='T4vsT6')
X64=schedule.addVars( days , vtype=GRB.BINARY, name='T6vsT4')

X47=schedule.addVars( days , vtype=GRB.BINARY, name='T4vsT7')
X74=schedule.addVars( days , vtype=GRB.BINARY, name='T7vsT4')

X48=schedule.addVars( days , vtype=GRB.BINARY, name='T4vsT8')
X84=schedule.addVars( days , vtype=GRB.BINARY, name='T8vsT4')

X56=schedule.addVars( days , vtype=GRB.BINARY, name='T5vsT6')
X65=schedule.addVars( days , vtype=GRB.BINARY, name='T6vsT5')

X57=schedule.addVars( days , vtype=GRB.BINARY, name='T5vsT7')
X75=schedule.addVars( days , vtype=GRB.BINARY, name='T7vsT5')

X58=schedule.addVars( days , vtype=GRB.BINARY, name='T5vsT8')
X85=schedule.addVars( days , vtype=GRB.BINARY, name='T8vsT5')

X67=schedule.addVars( days , vtype=GRB.BINARY, name='T6vsT7')
X76=schedule.addVars( days , vtype=GRB.BINARY, name='T7vsT6')

X68=schedule.addVars( days , vtype=GRB.BINARY, name='T6vsT8')
X86=schedule.addVars( days , vtype=GRB.BINARY, name='T8vsT6')

X78=schedule.addVars( days , vtype=GRB.BINARY, name='T7vsT8')
X87=schedule.addVars( days , vtype=GRB.BINARY, name='T8vsT7')



#Each team plays against each other team twice(once a home game and the other game away)
schedule.addConstr(sum(X12[i] for i in days) == 1)
schedule.addConstr(sum(X21[i] for i in days) == 1)

schedule.addConstr(sum(X13[i] for i in days) == 1)
schedule.addConstr(sum(X31[i] for i in days) == 1)

schedule.addConstr(sum(X14[i] for i in days) == 1)
schedule.addConstr(sum(X41[i] for i in days) == 1)

schedule.addConstr(sum(X15[i] for i in days) == 1)
schedule.addConstr(sum(X51[i] for i in days) == 1)

schedule.addConstr(sum(X16[i] for i in days) == 1)
schedule.addConstr(sum(X61[i] for i in days) == 1)

schedule.addConstr(sum(X17[i] for i in days) == 1)
schedule.addConstr(sum(X71[i] for i in days) == 1)

schedule.addConstr(sum(X18[i] for i in days) == 1)
schedule.addConstr(sum(X81[i] for i in days) == 1)

schedule.addConstr(sum(X23[i] for i in days) == 1)
schedule.addConstr(sum(X32[i] for i in days) == 1)

schedule.addConstr(sum(X24[i] for i in days) == 1)
schedule.addConstr(sum(X42[i] for i in days) == 1)

schedule.addConstr(sum(X25[i] for i in days) == 1)
schedule.addConstr(sum(X52[i] for i in days) == 1)

schedule.addConstr(sum(X26[i] for i in days) == 1)
schedule.addConstr(sum(X62[i] for i in days) == 1)

schedule.addConstr(sum(X27[i] for i in days) == 1)
schedule.addConstr(sum(X72[i] for i in days) == 1)

schedule.addConstr(sum(X28[i] for i in days) == 1)
schedule.addConstr(sum(X82[i] for i in days) == 1)

schedule.addConstr(sum(X34[i] for i in days) == 1)
schedule.addConstr(sum(X43[i] for i in days) == 1)

schedule.addConstr(sum(X35[i] for i in days) == 1)
schedule.addConstr(sum(X53[i] for i in days) == 1)

schedule.addConstr(sum(X36[i] for i in days) == 1)
schedule.addConstr(sum(X63[i] for i in days) == 1)

schedule.addConstr(sum(X37[i] for i in days) == 1)
schedule.addConstr(sum(X73[i] for i in days) == 1)

schedule.addConstr(sum(X38[i] for i in days) == 1)
schedule.addConstr(sum(X83[i] for i in days) == 1)

schedule.addConstr(sum(X45[i] for i in days) == 1)
schedule.addConstr(sum(X54[i] for i in days) == 1)

schedule.addConstr(sum(X46[i] for i in days) == 1)
schedule.addConstr(sum(X64[i] for i in days) == 1)

schedule.addConstr(sum(X47[i] for i in days) == 1)
schedule.addConstr(sum(X74[i] for i in days) == 1)

schedule.addConstr(sum(X48[i] for i in days) == 1)
schedule.addConstr(sum(X84[i] for i in days) == 1)

schedule.addConstr(sum(X56[i] for i in days) == 1)
schedule.addConstr(sum(X65[i] for i in days) == 1)

schedule.addConstr(sum(X57[i] for i in days) == 1)
schedule.addConstr(sum(X75[i] for i in days) == 1)

schedule.addConstr(sum(X58[i] for i in days) == 1)
schedule.addConstr(sum(X85[i] for i in days) == 1)

schedule.addConstr(sum(X67[i] for i in days) == 1)
schedule.addConstr(sum(X76[i] for i in days) == 1)

schedule.addConstr(sum(X68[i] for i in days) == 1)
schedule.addConstr(sum(X86[i] for i in days) == 1)

schedule.addConstr(sum(X78[i] for i in days) == 1)
schedule.addConstr(sum(X87[i] for i in days) == 1)

#Each day 4 games are played
schedule.addConstrs(X12[i]+X21[i]+X13[i]+X31[i]+X14[i]+X41[i]+X15[i]+X51[i]+X16[i]+X61[i]+X17[i]+X71[i]+X18[i]+X81[i] \
             +X23[i]+X32[i]+X24[i]+X42[i]+X25[i]+X52[i]+X26[i]+X62[i]+X27[i]+X72[i]+X28[i]+X82[i] \
             +X34[i]+X43[i]+X35[i]+X53[i]+X36[i]+X63[i]+X37[i]+X73[i]+X38[i]+X83[i] \
             +X45[i]+X54[i]+X46[i]+X64[i]+X47[i]+X74[i]+X48[i]+X84[i] \
             +X56[i]+X65[i]+X57[i]+X75[i]+X58[i]+X85[i] \
             +X67[i]+X76[i]+X68[i]+X86[i] \
             +X78[i]+X87[i]   == 4 for i in days )

#Each team can only play once a day
schedule.addConstrs(X12[i]+X21[i]+X13[i]+X31[i]+X14[i]+X41[i]+X15[i]+X51[i]+X16[i]+X61[i]+X17[i]+X71[i]+X18[i]+X81[i] == 1 for i in days )

schedule.addConstrs(X21[i]+X12[i]+X23[i]+X32[i]+X24[i]+X42[i]+X25[i]+X52[i]+X26[i]+X62[i]+X27[i]+X72[i]+X28[i]+X82[i] == 1 for i in days )

schedule.addConstrs(X13[i]+X31[i]+X32[i]+X23[i]+X34[i]+X43[i]+X35[i]+X53[i]+X36[i]+X63[i]+X37[i]+X73[i]+X38[i]+X83[i] == 1 for i in days )

schedule.addConstrs(X14[i]+X41[i]+X24[i]+X42[i]+X34[i]+X43[i]+X45[i]+X54[i]+X46[i]+X64[i]+X47[i]+X74[i]+X48[i]+X84[i] == 1 for i in days )

schedule.addConstrs(X15[i]+X51[i]+X25[i]+X52[i]+X35[i]+X53[i]+X45[i]+X54[i]+X56[i]+X65[i]+X57[i]+X75[i]+X58[i]+X85[i] == 1 for i in days )

schedule.addConstrs(X16[i]+X61[i]+X26[i]+X62[i]+X36[i]+X63[i]+X46[i]+X64[i]+X56[i]+X65[i]+X67[i]+X76[i]+X68[i]+X86[i] == 1 for i in days )

schedule.addConstrs(X71[i]+X17[i]+X72[i]+X27[i]+X73[i]+X37[i]+X74[i]+X47[i]+X75[i]+X57[i]+X76[i]+X67[i]+X78[i]+X87[i] == 1 for i in days )

schedule.addConstrs(X81[i]+X18[i]+X82[i]+X28[i]+X83[i]+X38[i]+X84[i]+X48[i]+X85[i]+X58[i]+X86[i]+X68[i]+X87[i]+X78[i] == 1 for i in days )


#Each two teams play in the same day of each respective round
schedule.addConstrs(X12[i] == X21[i+7] for i in range(len(days) // 2) )
schedule.addConstrs(X21[i] == X12[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X13[i] == X31[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X31[i] == X13[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X14[i] == X41[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X41[i] == X14[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X15[i] == X51[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X51[i] == X15[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X16[i] == X61[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X61[i] == X16[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X17[i] == X71[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X71[i] == X17[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X18[i] == X81[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X81[i] == X18[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X23[i] == X32[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X32[i] == X23[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X24[i] == X42[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X42[i] == X24[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X25[i] == X52[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X52[i] == X25[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X26[i] == X62[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X62[i] == X26[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X27[i] == X72[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X72[i] == X27[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X28[i] == X82[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X82[i] == X28[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X34[i] == X43[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X43[i] == X34[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X35[i] == X53[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X53[i] == X35[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X36[i] == X63[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X63[i] == X36[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X37[i] == X73[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X73[i] == X37[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X38[i] == X83[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X83[i] == X38[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X45[i] == X54[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X54[i] == X45[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X46[i] == X64[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X64[i] == X46[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X47[i] == X74[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X74[i] == X47[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X48[i] == X84[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X84[i] == X48[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X56[i] == X65[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X65[i] == X56[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X57[i] == X75[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X75[i] == X57[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X58[i] == X85[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X85[i] == X58[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X67[i] == X76[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X76[i] == X67[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X68[i] == X86[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X86[i] == X68[i+7] for i in range(len(days)//2) )

schedule.addConstrs(X87[i] == X78[i+7] for i in range(len(days)//2) )
schedule.addConstrs(X78[i] == X87[i+7] for i in range(len(days)//2) )


#Sum of two successive home games initialized
sumhome=0


#Sum of two successive away games initialized
sumaway=0

#Sum of all possible successive home/away games
#in the optimal case, the multiplication in each line is equal to zero (Otherwise it is 1)

for i in range((len(days)//2)-1):
    j=i+7

    sumhome+=(X12[i]+X13[i]+X14[i]+X15[i]+X16[i]+X17[i]+X18[i]) * \
             (X12[i+1]+X13[i+1]+X14[i+1]+X15[i+1]+X16[i+1]+X17[i+1]+X18[i+1])
    sumhome+=(X12[j]+X13[j]+X14[j]+X15[j]+X16[j]+X17[j]+X18[j]) * \
             (X12[j+1]+X13[j+1]+X14[j+1]+X15[j+1]+X16[j+1]+X17[j+1]+X18[j+1])

    sumhome+=(X21[i]+X23[i]+X24[i]+X25[i]+X26[i]+X27[i]+X28[i]) * \
             (X21[i+1]+X23[i+1]+X24[i+1]+X25[i+1]+X26[i+1]+X27[i+1]+X28[i+1])
    sumhome+=(X21[j]+X23[j]+X24[j]+X25[j]+X26[j]+X27[j]+X28[j]) * \
             (X21[j+1]+X23[j+1]+X24[j+1]+X25[j+1]+X26[j+1]+X27[j+1]+X28[j+1])

    sumhome+=(X31[i]+X32[i]+X34[i]+X35[i]+X36[i]+X37[i]+X38[i]) * \
             (X31[i+1]+X32[i+1]+X34[i+1]+X35[i+1]+X36[i+1]+X37[i+1]+X38[i+1])
    sumhome+=(X31[j]+X32[j]+X34[j]+X35[j]+X36[j]+X37[j]+X38[j]) * \
             (X31[j+1]+X32[j+1]+X34[j+1]+X35[j+1]+X36[j+1]+X37[j+1]+X38[j+1])

    sumhome+=(X41[i]+X42[i]+X43[i]+X45[i]+X46[i]+X47[i]+X48[i]) * \
       (X41[i+1]+X42[i+1]+X43[i+1]+X45[i+1]+X46[i+1]+X47[i+1]+X48[i+1])
    sumhome+=(X41[j]+X42[j]+X43[j]+X45[j]+X46[j]+X47[j]+X48[j]) * \
       (X41[j+1]+X42[j+1]+X43[j+1]+X45[j+1]+X46[j+1]+X47[j+1]+X48[j+1])

    sumhome+=(X51[i]+X52[i]+X53[i]+X54[i]+X56[i]+X57[i]+X58[i]) * \
       (X51[i+1]+X52[i+1]+X53[i+1]+X54[i+1]+X56[i+1]+X57[i+1]+X58[i+1])
    sumhome+=(X51[j]+X52[j]+X53[j]+X54[j]+X56[j]+X57[j]+X58[j]) * \
       (X51[j+1]+X52[j+1]+X53[j+1]+X54[j+1]+X56[j+1]+X57[j+1]+X58[j+1])

    sumhome+=(X61[i]+X62[i]+X63[i]+X64[i]+X65[i]+X67[i]+X68[i]) * \
       (X61[i+1]+X62[i+1]+X63[i+1]+X64[i+1]+X65[i+1]+X67[i+1]+X68[i+1])
    sumhome+=(X61[j]+X62[j]+X63[j]+X64[j]+X65[j]+X67[j]+X68[j]) * \
       (X61[j+1]+X62[j+1]+X63[j+1]+X64[j+1]+X65[j+1]+X67[j+1]+X68[j+1])

    sumhome+=(X71[i]+X72[i]+X73[i]+X74[i]+X75[i]+X76[i]+X78[i]) * \
       (X71[i+1]+X72[i+1]+X73[i+1]+X74[i+1]+X75[i+1]+X76[i+1]+X78[i+1])
    sumhome+=(X71[j]+X72[j]+X73[j]+X74[j]+X75[j]+X76[j]+X78[j]) * \
       (X71[j+1]+X72[j+1]+X73[j+1]+X74[j+1]+X75[j+1]+X76[j+1]+X78[j+1])

    sumhome+=(X81[i]+X82[i]+X83[i]+X84[i]+X85[i]+X86[i]+X87[i]) * \
       (X81[i+1]+X82[i+1]+X83[i+1]+X84[i+1]+X85[i+1]+X86[i+1]+X87[i+1])
    sumhome+=(X81[j]+X82[j]+X83[j]+X84[j]+X85[j]+X86[j]+X87[j]) * \
       (X81[j+1]+X82[j+1]+X83[j+1]+X84[j+1]+X85[j+1]+X86[j+1]+X87[j+1])


    sumaway+=(X21[i]+X31[i]+X41[i]+X51[i]+X61[i]+X71[i]+X81[i]) * \
       (X21[i+1]+X31[i+1]+X41[i+1]+X51[i+1]+X61[i+1]+X71[i+1]+X81[i+1])
    sumaway+=(X21[j]+X31[j]+X41[j]+X51[j]+X61[j]+X71[j]+X81[j]) * \
       (X21[j+1]+X31[j+1]+X41[j+1]+X51[j+1]+X61[j+1]+X71[j+1]+X81[j+1])

    sumaway+=(X12[i]+X32[i]+X42[i]+X52[i]+X62[i]+X72[i]+X82[i]) * \
       (X12[i+1]+X32[i+1]+X42[i+1]+X52[i+1]+X62[i+1]+X72[i+1]+X82[i+1])
    sumaway+=(X12[j]+X32[j]+X42[j]+X52[j]+X62[j]+X72[j]+X82[j]) * \
       (X12[j+1]+X32[j+1]+X42[j+1]+X52[j+1]+X62[j+1]+X72[j+1]+X82[j+1])

    sumaway+=(X13[i]+X23[i]+X43[i]+X53[i]+X63[i]+X73[i]+X83[i]) * \
       (X13[i+1]+X23[i+1]+X43[i+1]+X53[i+1]+X63[i+1]+X73[i+1]+X83[i+1])
    sumaway+=(X13[j]+X23[j]+X43[j]+X53[j]+X63[j]+X73[j]+X83[j]) * \
       (X13[j+1]+X23[j+1]+X43[j+1]+X53[j+1]+X63[j+1]+X73[j+1]+X83[j+1])

    sumaway+=(X14[i]+X24[i]+X34[i]+X54[i]+X64[i]+X74[i]+X84[i]) * \
       (X14[i+1]+X24[i+1]+X34[i+1]+X54[i+1]+X64[i+1]+X74[i+1]+X84[i+1])
    sumaway+=(X14[j]+X24[j]+X34[j]+X54[j]+X64[j]+X74[j]+X84[j]) * \
       (X14[j+1]+X24[j+1]+X34[j+1]+X54[j+1]+X64[j+1]+X74[j+1]+X84[j+1])

    sumaway+=(X15[i]+X25[i]+X35[i]+X45[i]+X65[i]+X75[i]+X85[i]) * \
       (X15[i+1]+X25[i+1]+X35[i+1]+X45[i+1]+X65[i+1]+X75[i+1]+X85[i+1])
    sumaway+=(X15[j]+X25[j]+X35[j]+X45[j]+X65[j]+X75[j]+X85[j]) * \
       (X15[j+1]+X25[j+1]+X35[j+1]+X45[j+1]+X65[j+1]+X75[j+1]+X85[j+1])

    sumaway+=(X16[i]+X26[i]+X36[i]+X46[i]+X56[i]+X76[i]+X86[i]) * \
       (X16[i+1]+X26[i+1]+X36[i+1]+X46[i+1]+X56[i+1]+X76[i+1]+X86[i+1])
    sumaway+=(X16[j]+X26[j]+X36[j]+X46[j]+X56[j]+X76[j]+X86[j]) * \
       (X16[j+1]+X26[j+1]+X36[j+1]+X46[j+1]+X56[j+1]+X76[j+1]+X86[j+1])

    sumaway+=(X17[i]+X27[i]+X37[i]+X47[i]+X57[i]+X67[i]+X87[i]) * \
       (X17[i+1]+X27[i+1]+X37[i+1]+X47[i+1]+X57[i+1]+X67[i+1]+X87[i+1])
    sumaway+=(X17[j]+X27[j]+X37[j]+X47[j]+X57[j]+X67[j]+X87[j]) * \
       (X17[j+1]+X27[j+1]+X37[j+1]+X47[j+1]+X57[j+1]+X67[j+1]+X87[j+1])

    sumaway+=(X18[i]+X28[i]+X38[i]+X48[i]+X58[i]+X68[i]+X78[i]) * \
       (X18[i+1]+X28[i+1]+X38[i+1]+X48[i+1]+X58[i+1]+X68[i+1]+X78[i+1])
    sumaway+=(X18[j]+X28[j]+X38[j]+X48[j]+X58[j]+X68[j]+X78[j]) * \
       (X18[j+1]+X28[j+1]+X38[j+1]+X48[j+1]+X58[j+1]+X68[j+1]+X78[j+1])

#Objective function
Obj=sumhome+sumaway

schedule.setObjective(Obj, GRB.MINIMIZE)
schedule.setParam('TimeLimit', 200)

schedule.optimize()